# Book Inventory System

### Importing Required Modules

In [3]:
import json
import tkinter as tk
from tkinter import messagebox

### Book Class

In [4]:
class Book:
    def __init__(self, title, author, year, isbn):
        self.title = title
        self.author = author
        self.year = year
        self.isbn = isbn

    def __str__(self):
        return f"{self.title}, by {self.author} ({self.year}) - ISBN: {self.isbn}"


### Inventory class

In [5]:
class Inventory:
    def __init__(self, file_name="books.json"):
        self.file_name = file_name
        self.books = []
        self.load_books()

    def add_book(self, book):
        self.books.append(book)
        self.save_books()

    def update_book(self, index, book):
        if 0 <= index < len(self.books):
            self.books[index] = book
            self.save_books()

    def delete_book(self, index):
        if 0 <= index < len(self.books):
            del self.books[index]
            self.save_books()

    def search_books(self, title):
        return [book for book in self.books if title.lower() in book.title.lower()]

    def save_books(self):
        with open(self.file_name, 'w') as f:
            json.dump([book.__dict__ for book in self.books], f)

    def load_books(self):
        try:
            with open(self.file_name, 'r') as f:
                books_data = json.load(f)
                self.books = [Book(**data) for data in books_data]
        except FileNotFoundError:
            self.books = []


### Tkinter GUI

In [6]:
class BookInventoryApp:
    def __init__(self, root, inventory):
        self.root = root
        self.inventory = inventory
        self.root.title("Book Inventory System")

        # Widgets
        self.title_label = tk.Label(root, text="Title")
        self.title_label.grid(row=0, column=0)
        self.title_entry = tk.Entry(root)
        self.title_entry.grid(row=0, column=1)

        self.author_label = tk.Label(root, text="Author")
        self.author_label.grid(row=1, column=0)
        self.author_entry = tk.Entry(root)
        self.author_entry.grid(row=1, column=1)

        self.year_label = tk.Label(root, text="Year")
        self.year_label.grid(row=2, column=0)
        self.year_entry = tk.Entry(root)
        self.year_entry.grid(row=2, column=1)

        self.isbn_label = tk.Label(root, text="ISBN")
        self.isbn_label.grid(row=3, column=0)
        self.isbn_entry = tk.Entry(root)
        self.isbn_entry.grid(row=3, column=1)

        self.add_button = tk.Button(root, text="Add Book", command=self.add_book)
        self.add_button.grid(row=4, column=0, columnspan=2)

        self.books_listbox = tk.Listbox(root)
        self.books_listbox.grid(row=5, column=0, columnspan=2)
        self.books_listbox.bind('<<ListboxSelect>>', self.load_selected_book)

        self.update_button = tk.Button(root, text="Update Book", command=self.update_book)
        self.update_button.grid(row=6, column=0, columnspan=2)

        self.delete_button = tk.Button(root, text="Delete Book", command=self.delete_book)
        self.delete_button.grid(row=7, column=0, columnspan=2)

        self.search_label = tk.Label(root, text="Search by Title")
        self.search_label.grid(row=8, column=0)
        self.search_entry = tk.Entry(root)
        self.search_entry.grid(row=8, column=1)
        self.search_button = tk.Button(root, text="Search", command=self.search_books)
        self.search_button.grid(row=9, column=0, columnspan=2)

        self.populate_listbox()

    def populate_listbox(self):
        self.books_listbox.delete(0, tk.END)
        for book in self.inventory.books:
            self.books_listbox.insert(tk.END, book)

    def load_selected_book(self, event):
        try:
            index = self.books_listbox.curselection()[0]
            selected_book = self.inventory.books[index]
            self.title_entry.delete(0, tk.END)
            self.title_entry.insert(tk.END, selected_book.title)
            self.author_entry.delete(0, tk.END)
            self.author_entry.insert(tk.END, selected_book.author)
            self.year_entry.delete(0, tk.END)
            self.year_entry.insert(tk.END, selected_book.year)
            self.isbn_entry.delete(0, tk.END)
            self.isbn_entry.insert(tk.END, selected_book.isbn)
        except IndexError:
            pass

    def add_book(self):
        book = Book(
            self.title_entry.get(),
            self.author_entry.get(),
            self.year_entry.get(),
            self.isbn_entry.get()
        )
        self.inventory.add_book(book)
        self.populate_listbox()
        self.clear_entries()

    def update_book(self):
        try:
            index = self.books_listbox.curselection()[0]
            book = Book(
                self.title_entry.get(),
                self.author_entry.get(),
                self.year_entry.get(),
                self.isbn_entry.get()
            )
            self.inventory.update_book(index, book)
            self.populate_listbox()
            self.clear_entries()
        except IndexError:
            messagebox.showwarning("Update Book", "No book selected")

    def delete_book(self):
        try:
            index = self.books_listbox.curselection()[0]
            self.inventory.delete_book(index)
            self.populate_listbox()
            self.clear_entries()
        except IndexError:
            messagebox.showwarning("Delete Book", "No book selected")

    def search_books(self):
        self.books_listbox.delete(0, tk.END)
        for book in self.inventory.search_books(self.search_entry.get()):
            self.books_listbox.insert(tk.END, book)

    def clear_entries(self):
        self.title_entry.delete(0, tk.END)
        self.author_entry.delete(0, tk.END)
        self.year_entry.delete(0, tk.END)
        self.isbn_entry.delete(0, tk.END)

### Main Block

In [8]:
if __name__ == "__main__":
    root = tk.Tk()
    inventory = Inventory()
    app = BookInventoryApp(root, inventory)
    root.mainloop()